Taken from
* https://www.kaggle.com/code/nghiahoangtrung/hubmap-mmdet3-1-single-fold-inference
* https://www.kaggle.com/code/ammarnassanalhajali/hubmap-2023-mmdetection-3-1-training

In [ ]:
%%time
!pip install -qqq /kaggle/input/mmdetv3-env/archive/addict-2.4.0-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmengine-0.7.4-py3-none-any.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/mmcv-2.0.0-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetv3-env/archive/terminaltables-3.1.10-py2.py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/weighted-boxes-fusion/ensemble_boxes-1.0.9-py3-none-any.whl
!pip install -qqq /kaggle/input/pycocotools-206/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install -qqq /kaggle/input/mmdetection-3-1-evn/src/mmdet-3.1.0-py3-none-any.whl

In [ ]:
from itertools import groupby
from pycocotools import mask as mutils
from pycocotools.coco import COCO
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import wandb
from PIL import Image
import gc

from glob import glob
import matplotlib.pyplot as plt

import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import Text, Dict, Tuple
import zlib

import mmdet, mmcv, mmengine
from mmengine.config import Config
from mmengine.runner import Runner
from mmdet.utils import register_all_modules
from mmdet.apis import init_detector, inference_detector
from mmengine.visualization import Visualizer


print(mmdet.__version__)
print(mmcv.__version__)
print(mmengine.__version__)

In [ ]:
from mmengine.config import Config
base="mask-rcnn_r50_fpn_2x_coco"
base1="mask-rcnn_x101-64x4d_fpn_2x_coco"
cfg = Config.fromfile(f'/kaggle/input/mmdet-github/mmdetection-main/configs/mask_rcnn/{base}.py')

cfg.load_from = 'https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_fpn_2x_coco/mask_rcnn_r50_fpn_2x_coco_bbox_mAP-0.392__segm_mAP-0.354_20200505_003907-3e542a40.pth'
#cfg.load_from  =None
#----------------------------------------------------
width=512
height=512

max_epochs=20

batch_size=6
num_classes=2

cfg.work_dir = '/kaggle/working/'
dataset_type = 'CocoDataset' 
classes = ('blood_vessel','glomerulus') 
data_root = '/kaggle/input/hubmap-hacking-the-human-vasculature/'
#-----------------------------------------------------
cfg.model.roi_head.bbox_head.num_classes = num_classes
cfg.model.roi_head.mask_head.num_classes = num_classes
#-----------------------------------------------------
cfg.train_pipeline[2]['scale']=(width,height)
cfg.test_pipeline [1]['scale']=(width,height)
#-----------------------------------------------------
cfg.train_dataloader.dataset.type=dataset_type
cfg.train_dataloader.dataset.metainfo=dict(classes=classes)
cfg.train_dataloader.dataset.data_root=data_root
cfg.train_dataloader.dataset.ann_file='/kaggle/input/coco-dataset-hubmap-2023/coco_annotations_train_all_folds5_fold2.json'
cfg.train_dataloader.dataset.data_prefix=dict(img='train/')
cfg.train_dataloader.batch_size=batch_size
cfg.train_dataloader.dataset.pipeline[2]['scale']=(width,height)
#-----------------------------------------------------
cfg.val_dataloader.dataset.type=dataset_type
cfg.val_dataloader.dataset.metainfo=dict(classes=classes)
cfg.val_dataloader.dataset.data_root=data_root
cfg.val_dataloader.dataset.ann_file='/kaggle/input/coco-dataset-hubmap-2023/coco_annotations_valid_all_folds5_fold2.json'
cfg.val_dataloader.dataset.data_prefix=dict(img='train/')
cfg.val_dataloader.batch_size=batch_size
cfg.val_dataloader.dataset.pipeline[1]['scale']=(width,height)
#-----------------------------------------------------
cfg.test_dataloader.dataset.type=dataset_type
cfg.test_dataloader.dataset.metainfo=dict(classes=classes)
cfg.test_dataloader.dataset.data_root=data_root
cfg.test_dataloader.dataset.ann_file='/kaggle/input/coco-dataset-hubmap-2023/coco_annotations_valid_all_folds5_fold2.json'
cfg.test_dataloader.dataset.data_prefix=dict(img='train/')
cfg.test_dataloader.dataset.pipeline[1]['scale']=(width,height)
#------------------------------------------------------
#------------------------------------------------------
cfg.val_evaluator.type='CocoMetric'
cfg.val_evaluator.ann_file='/kaggle/input/coco-dataset-hubmap-2023/coco_annotations_valid_all_folds5_fold2.json'
cfg.val_evaluator.metric=['segm']

cfg.test_evaluator.type='CocoMetric'
cfg.test_evaluator.ann_file='/kaggle/input/coco-dataset-hubmap-2023/coco_annotations_valid_all_folds5_fold2.json'
cfg.test_evaluator.metric=['segm']
#------------------------------------------------------
cfg.train_cfg.max_epochs=max_epochs
cfg.optim_wrapper.type='OptimWrapper'
cfg.optim_wrapper.optimizer=dict(type='AdamW',lr=0.001,weight_decay=0.05,eps=1e-8,betas=(0.9, 0.999))
cfg.default_hooks = dict(logger=dict(type='LoggerHook', interval=200),
                         checkpoint=dict(type='CheckpointHook', interval=1, save_best='coco/segm_mAP'))
#------------------------------------------------------
!mkdir -p configs/HuBMAP
config=f'configs/HuBMAP/custom_config_{base}_{width}_{height}.py'
with open(config, 'w') as f:
    f.write(cfg.pretty_text)

In [ ]:
from mmengine.config import Config
from mmengine.runner import Runner

cfg = Config.fromfile(config)
runner = Runner.from_cfg(cfg)

runner.train()